In [2]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [3]:
# Read the data from CSV files
temp = pd.read_csv('../For_preprocessing.csv')
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4768 entries, 0 to 4767
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4763 non-null   object
 1   Sentiment    4766 non-null   object
dtypes: object(2)
memory usage: 74.6+ KB


In [4]:
temp=temp.dropna(subset=['Review_Text'])
temp=temp.dropna(subset=['Sentiment'])
temp = temp.reset_index(drop=True)
temp_val = temp['Sentiment'].tolist()
#len(temp_val)
ind_list = []

for i in range(len(temp['Sentiment'])):
    if temp_val[i] != '1' and temp_val[i] != '2' and temp_val[i] != '3':
        ind_list.append(i)
temp.reset_index(drop=True)
temp.drop(temp.index[ind_list], inplace=True)

In [5]:
temp = temp.reset_index(drop=True)
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4699 entries, 0 to 4698
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4699 non-null   object
 1   Sentiment    4699 non-null   object
dtypes: object(2)
memory usage: 73.5+ KB


In [6]:
def tokenize_reg():
    temp['Review_Text'] = temp['Review_Text'].replace(r'http\S+','',regex=True).replace(r'www\S+','',regex=True).replace(r'\d+','',regex=True)
    tokens = RegexpTokenizer(r'\w+')
    temp['Review_Text']=temp['Review_Text'].apply(lambda x:tokens.tokenize(x.lower()))
    stop_words = set(stopwords.words('english')) 
    stop_words = stop_words.union(",","(",")","[","]","{","}","#","@","!",":",";",".","?")
    temp['Review_Text'] = temp['Review_Text'].apply(lambda x: [item for item in x if item not in stop_words])
tokenize_reg()

In [7]:
lem = WordNetLemmatizer()
def lemm(text):
    sent=[]
    for word in text:
        sent.append(lem.lemmatize(word))
    return sent

ps=PorterStemmer()

def stemm(text):        
    sent = []
    for word in text:
        sent.append(ps.stem(word))
    return sent

temp['Review_Text'] =  temp.apply(lambda x: stemm(lemm(x['Review_Text'])), axis=1)

In [8]:
revs=temp['Review_Text'].tolist()
for i in range(len(revs)):
    revs[i]=' '.join(revs[i])
temp['Review_Text'] = revs
temp

,Review_Text,Sentiment
0,realli nice place stay especi busi tourist purpos,3
1,seem hotel check basic amen room hand room tra...,1
2,worst hotel ever encount never think stay thii...,1
3,good time hotel staff kumar aishwarya hous kee...,3
4,good hotel staff veg food good non veg breakfa...,3
...,...,...
4694,fifth stay hotel busi room great restaur excel...,3
4695,enjoy,3
4696,impress servic staff area good restaur fit cen...,3
4697,linen smell bad elev pungent odour housekeep p...,1


In [9]:
st = temp['Sentiment'].tolist()
st=set(st)

In [10]:
scale_mapper3 = {"1":0, "2":1, "3":2}
temp['Sentiment'] = temp['Sentiment'].replace(scale_mapper3)

In [11]:
temp = temp.reset_index(drop=True)
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4699 entries, 0 to 4698
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4699 non-null   object
 1   Sentiment    4699 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 73.5+ KB


In [12]:
temp

,Review_Text,Sentiment
0,realli nice place stay especi busi tourist purpos,2
1,seem hotel check basic amen room hand room tra...,0
2,worst hotel ever encount never think stay thii...,0
3,good time hotel staff kumar aishwarya hous kee...,2
4,good hotel staff veg food good non veg breakfa...,2
...,...,...
4694,fifth stay hotel busi room great restaur excel...,2
4695,enjoy,2
4696,impress servic staff area good restaur fit cen...,2
4697,linen smell bad elev pungent odour housekeep p...,0


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier


text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(random_state=0, class_weight='balanced'))])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__loss': ['log'],
    'clf__penalty': ['elasticnet'],
    'clf__alpha': [10 ** x for x in range(-6, 1)],
    'clf__l1_ratio': [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1]
}

The dataset was plited into train and test subsets.

In [10]:
x_train, x_test, y_train, y_test = train_test_split(temp['Review_Text'], temp['Sentiment'], test_size=0.33, random_state=42)

In [11]:
# from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

# score = 'f1_macro'
# print("# Tuning hyper-parameters for %s" % score)
# print()
# np.errstate(divide='ignore')
# clf = GridSearchCV(text_clf, tuned_parameters, cv=10, scoring=score,n_jobs=4)
# clf.fit(x_train, y_train)

# print("Best parameters set found on development set:")
# print()
# print(clf.best_params_)
# print()
# print("Grid scores on development set:")
# print()
# for mean, std, params in zip(clf.cv_results_['mean_test_score'], 
#                              clf.cv_results_['std_test_score'], 
#                              clf.cv_results_['params']):
#     print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
# print()

# print("Detailed classification report:")
# print()
# print("The model is trained on the full development set.")
# print("The scores are computed on the full evaluation set.")
# print()
# print(classification_report(y_test, clf.predict(x_test), digits=4))
# print()
# print ("Confusion Matrix")
# print(confusion_matrix(y_test, clf.predict(x_test)))
# print("Accuracy Score")
# print(accuracy_score(y_test, clf.predict(x_test)))

In [12]:
grid = GridSearchCV(text_clf,tuned_parameters,refit = True, verbose=2, n_jobs = 4)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 756 candidates, totalling 3780 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    4.4s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   14.1s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:   30.6s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:   54.6s
[Parallel(n_jobs=4)]: Done 1005 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 1450 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done 1977 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done 2584 tasks      | elapsed:  3.4min
[Parallel(n_jobs=4)]: Done 3273 tasks      | elapsed:  4.3min
[Parallel(n_jobs=4)]: Done 3780 out of 3780 | elapsed:  4.9min finished


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        SGDClassifier(class_weight='balanced',
                                                      random_state=0))]),
             n_jobs=4,
             param_grid={'clf__alpha': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1,
                                        1],
                         'clf__l1_ratio': [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9,
                                           0.95, 1],
                         'clf__loss': ['log'], 'clf__penalty': ['elasticnet'],
                         'tfidf__norm': ('l1', 'l2'),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2), (2, 2)]},
             verbose=2)

In [13]:
print("Best parameters set found on development set:")
print()
print(grid.best_params_)


Best parameters set found on development set:

{'clf__alpha': 1e-06, 'clf__l1_ratio': 0, 'clf__loss': 'log', 'clf__penalty': 'elasticnet', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


In [20]:
best_clf = Pipeline([('vect', CountVectorizer(ngram_range= (1, 2))),
                     ('tfidf', TfidfTransformer(use_idf= True, norm= 'l2')),
                     ('clf', SGDClassifier(random_state=0, class_weight='balanced', 
                                 alpha = 1e-06, l1_ratio =  0, loss= 'log', 
                                 penalty =  'elasticnet'))])
best_clf.fit(x_train,y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=1e-06, class_weight='balanced', l1_ratio=0,
                               loss='log', penalty='elasticnet',
                               random_state=0))])

In [21]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, best_clf.predict(x_test), digits=4))
print()
print ("Confusion Matrix")
print(confusion_matrix(y_test, best_clf.predict(x_test)))
print("Accuracy Score")
print(accuracy_score(y_test, best_clf.predict(x_test)))


Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           1     0.6626    0.6667    0.6646       162
           2     0.5972    0.4760    0.5298       271
           3     0.8882    0.9311    0.9092      1118

    accuracy                         0.8240      1551
   macro avg     0.7160    0.6913    0.7012      1551
weighted avg     0.8138    0.8240    0.8173      1551


Confusion Matrix
[[ 108   25   29]
 [  40  129  102]
 [  15   62 1041]]
Accuracy Score
0.8239845261121856


In [16]:
wo_hyp_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(random_state=0, class_weight='balanced'))])

wo_hyp_clf.fit(x_train,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(class_weight='balanced', random_state=0))])

In [17]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, wo_hyp_clf.predict(x_test), digits=4))
print()
print ("Confusion Matrix")
print(confusion_matrix(y_test, wo_hyp_clf.predict(x_test)))
print("Accuracy Score")
print(accuracy_score(y_test, wo_hyp_clf.predict(x_test)))


Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           1     0.6228    0.6420    0.6322       162
           2     0.5520    0.5092    0.5298       271
           3     0.8924    0.9052    0.8988      1118

    accuracy                         0.8085      1551
   macro avg     0.6891    0.6855    0.6869      1551
weighted avg     0.8048    0.8085    0.8064      1551


Confusion Matrix
[[ 104   28   30]
 [  41  138   92]
 [  22   84 1012]]
Accuracy Score
0.8085106382978723
